<a href="https://colab.research.google.com/github/benjaminBoboul/FashionConvNet/blob/master/FashionTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup directory

cloning and merging directories together.

In [0]:
!git clone https://github.com/aryapei/In-shop-Clothes-From-Deepfashion.git
!rsync -a ./In-shop-Clothes-From-Deepfashion/Img/MEN/ ./In-shop-Clothes-From-Deepfashion/Img/WOMEN/

# Define Neural Network

Create a ConvNet instance and remove last layer to implement transfert learning.

:warning: do not forget to freeze pretrained model reduce training workload.

In [0]:
%tensorflow_version 2.x
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import subprocess
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Flatten, Dropout, Dense, GlobalAveragePooling2D
from tensorflow.keras import losses

head = InceptionResNetV2(input_shape=(200, 200, 3), weights="imagenet", include_top=False)

model = Sequential()
model.add(head)
model.add(GlobalAveragePooling2D())
model.add(Dense(512,activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(17, activation="softmax"))

print(f"Number of trainable weights before freezing the head base: {len(model.trainable_weights)}.")
head.trainable = False

model.compile(loss="categorical_crossentropy", optimizer=RMSprop(lr=1e-4), metrics=["accuracy"])

print(f"Number of trainable weights after freezing the head base: {len(model.trainable_weights)}.")

model.summary()

for layer in model.layers:
    print(layer, layer.trainable)

# Train our neural network

We're using the dataset from https://github.com/aryapei/In-shop-Clothes-From-Deepfashion/ to train our neural network with flow_from_directory feature.

In [0]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2)

batch_size = 8

print("Generating train dataset")
train_generator = train_datagen.flow_from_directory("./In-shop-Clothes-From-Deepfashion/Img/WOMEN/",
                                                    batch_size=batch_size,
                                                    target_size=(200, 200),
                                                    class_mode="categorical",
                                                    subset="training")

print(f"Labels: {train_generator.class_indices}")

print("Generating validation dataset")
validation_generator = train_datagen.flow_from_directory("./In-shop-Clothes-From-Deepfashion/Img/WOMEN/",
                                                         batch_size=batch_size,
                                                         target_size=(200, 200),
                                                         class_mode="categorical",
                                                         subset="validation")

model.fit(train_generator,
          validation_data=validation_generator,
          validation_steps=validation_generator.samples // batch_size,
          steps_per_epoch=train_generator.samples // batch_size,
          use_multiprocessing=False,
          epochs=100)

model.save('plankton_mind.h5')